In [ ]:
import os
import tensorflow as tf
from tensorflow import keras


In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator 
# https://keras.io/api/preprocessing/image/#image-data-preprocessing
from tensorflow.keras.applications.imagenet_utils import preprocess_input
from tensorflow.keras.applications import InceptionV3, ResNet101V2
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv3D, MaxPool3D, Flatten, Activation, BatchNormalization
from tensorflow.keras.activations import relu, sigmoid

from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import binary_crossentropy
from tensorflow.keras.metrics import Accuracy as accuracy

import matplotlib.pyplot as plt

from tensorflow.keras import applications
from tensorflow.keras import optimizers
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Dropout, Flatten, Dense
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

In [ ]:
pathTrain = './../dataset/train'
pathTest = './../dataset/test'

if (os.path.exists(pathTrain) and os.path.exists(pathTest)):
    print('OK')
else:
    print('No existen carpetas')
    print('Descomprima el archivo catdog.zip o puede descargar otro dataset: https://www.kaggle.com/arpitjain007/dog-vs-cat-fastai')

In [ ]:
image_size = (224, 224)
images_increased = 5
width_shape = 224
height_shape = 224
batch_size = 32

train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    pathTrain,
    labels = "inferred",
    validation_split=0.2,
    subset="training",
    seed=1337,
    image_size=image_size,
    batch_size=batch_size,
)
val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    pathTest,
    labels= "inferred",
    validation_split=0.2,
    subset="validation",
    seed=1337,
    image_size=image_size,
    batch_size=batch_size,
)

In [ ]:
plt.figure(figsize=(10, 10))
for images, labels in train_ds.take(1):
    for i in range(9):
        ax = plt.subplot(3, 3, i + 1)
        plt.imshow(images[i].numpy().astype("uint8"))
        plt.title(int(labels[i]))
        plt.axis("off")

In [ ]:
num_classes = 275
convlayer=ResNet101V2(input_shape=(224,224,3),weights='imagenet',include_top=False)
for layer in convlayer.layers:
    layer.trainable=False
model = Sequential()
model.add(convlayer)
model.add(Dropout(0.5))
model.add(Flatten())
model.add(BatchNormalization())
model.add(Dense(2048,kernel_initializer='he_uniform'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1024,kernel_initializer='he_uniform'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(275,activation='softmax'))
model.summary()
model.compile(optimizer =optimizers.Adam(learning_rate=0.01),
               loss = 'sparse_categorical_crossentropy',
               metrics = ['accuracy'])

history = model.fit(
    train_ds,
    epochs=50,
    steps_per_epoch=50,
    validation_data=val_ds,
    validation_steps=20,
    verbose = 1,
    callbacks=[EarlyStopping(monitor='accuracy', patience=5, restore_best_weights= True)])

test_loss, test_acc = model.evaluate_generator(train_ds, steps = 20)
print('test_acc:', test_acc)

In [ ]:
history.history
plt.plot(range(len(history.history['loss'])), history.history['loss'], color='red')
plt.plot(range(len(history.history['val_loss'])), history.history['val_loss'], color='blue')
plt.show()